# Script de scrapping 

In [2]:
import requests as rq
from bs4 import BeautifulSoup
import pandas as pd

In [3]:
import numpy as np
from bs4 import UnicodeDammit

main_page = rq.get("http://www.autodescuento.com")
main_soup = BeautifulSoup(main_page.content, 'html.parser')
path_marcas = "http://www.autodescuento.com/coches-nuevos-"

lista_marcas = []

for marca in main_soup.find_all('h2', class_='h2'):
    
    lista_marcas.append(marca.get_text().strip().replace(' ','-'))

lista_marcas_limpia = lista_marcas[1:-2]


versiones_modelo = []

for marca in lista_marcas_limpia:
    
    second_page = rq.get(path_marcas+marca+".htm", headers={'User-Agent': 'Safari'})
    second_soup = BeautifulSoup(second_page.content, 'html.parser')
    #print('Otra marca')
    for modelo in second_soup.find('div', class_='izquierda').find_all('a', class_="coche redondas"):
        
        #print(modelo['href'])
        #model = modelo.find('p',class_='mm').getText().encode('utf-8') + "-"
        #print(type(model))
        third_page = rq.get(modelo['href'], headers={'User-Agent': 'Safari'})
        third_soup = BeautifulSoup(third_page.content, 'html.parser')
        
        if(third_soup.find('span',id='tdversiones') is not None):
            
            versiones = third_soup.find('span',id='tdversiones').find_all('a', href=True)

            for ver in versiones:

                forth_page = rq.get(ver.get('href').encode('utf-8'),headers={'User-Agent': 'Safari'})
                forth_soup = BeautifulSoup(forth_page.content, 'html.parser')
                
                #linea_modelo = model
                #print(linea_modelo)
                linea_modelo = forth_soup.find('h1').getText().encode('utf-8')+'-'+ver.find('div', class_='precio').getText().encode('utf-8')
    
                for campos in forth_soup.find_all('label', class_='campos'):
        
                    linea_modelo += "-"+campos.find_next('span').getText().encode('utf-8')        
                
                versiones_modelo.append(linea_modelo)
                                                                   
                                                                 

In [41]:
df = pd.DataFrame(versiones_modelo)

In [42]:
df.head(10)

,0
0,Abarth - 124 Spider - Abarth 1.4 170-35.093€-...
1,Abarth - 124 Spider - Abarth 1.4 Scorpione 17...
2,Abarth - 124 Spider - Abarth 1.4 Aut. 170-37....
3,Abarth - 124 Spider - Abarth 1.4 Scorpione Au...
4,Abarth - 595 - Abarth 1.4T-Jet 145-15.670€-3 ...
5,Abarth - 595 - Abarth C 1.4T-Jet 145-17.316€-2...
6,Abarth - 595 - Abarth 1.4T-Jet Turismo 165-20...
7,Abarth - 595 - Abarth 1.4T-Jet 145 Aut.-21.08...
8,Abarth - 595 - Abarth 1.4T-Jet Pista 160-21.8...
9,Abarth - 595 - Abarth C 1.4T-Jet Turismo 165-2...


In [65]:
df.to_csv('datos_crudo',encoding='utf-8',index=False)

In [66]:
!head prueba1

0
"Abarth - 124 Spider - Abarth  1.4 170-35.093€-2 puertas - -Gasolina -Tracción trasera -170 cv -Manual -6 marchas -148 gr/km -703,1 km -6,4 l/100km -0 -24 meses - -Gasolina -148 gr/km -1.368 cc -Tracción trasera -45 lts. -Manual -703,1 km -6 -Tracción trasera -232 km/h -6,80 s.-170 cv-6,4 l/100km -5000 rpm-250Nm -0 -Sí-No--- -No -No-No-No--No - --No/No/No-No-Roadster -4054/1740/1233-2-1135 kg"
"Abarth - 124 Spider - Abarth  1.4 Scorpione 170-36.511€-2 puertas - -Gasolina -Tracción trasera -170 cv -Manual -6 marchas -148 gr/km -703,1 km -6,4 l/100km -0 -24 meses - -Gasolina -148 gr/km -1.368 cc -Tracción trasera -45 lts. -Manual -703,1 km -6 -Tracción trasera -232 km/h -6,80 s.-170 cv-6,4 l/100km -5000 rpm-250Nm -0 -Sí-No--- -No -No-No-No--No - --No/No/No-No-Roadster -4054/1740/1233-2-1135 kg"
"Abarth - 124 Spider - Abarth  1.4 Aut. 170-37.961€-2 puertas - -Gasolina -Tracción trasera -170 cv -Automática secuencial -6 marchas -153 gr/km -681,8 km -6,6 l/100km -0 -24 meses - -Gasolina -

In [67]:
!sed 's/"//g' datos_crudo > datos_limpios

In [76]:
df_1 = pd.read_csv('datos_limpios', sep='-',names=colnames)

In [77]:
df_1 = df_1.drop(0)

In [78]:
df_1.head()

,marca,modelo,version,precio,puertas,desc,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,desc4,comb,co2.1,cilindrada,traccion.1,capacidad deposito,cambio.1,autonomia.1,marchas,traccion.2,max_vel,0_a_100,cv.1,consumo.1,rpm,par,airbags.1,control traccion,control estabilidad,estrellas NCAP,desc9,inmovilizador,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,desc16,n_altavoces,desc18,CD/mp3/dvd,cargador,carroceria,medidas largo/ancho/alto,puertas.1,peso,desc24
1,Abarth,124 Spider,Abarth 1.4 170,35.093€,2 puertas,,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,,Gasolina,148 gr/km,1.368 cc,Tracción trasera,45 lts.,Manual,"703,1 km",6,Tracción trasera,232 km/h,"6,80 s.",170 cv,"6,4 l/100km",5000 rpm,250Nm,0,Sí,No,NaN,NaN,,No,No,No,No,NaN,No,,NaN,No/No/No,No,Roadster,4054/1740/1233,2,1135 kg,NaN
2,Abarth,124 Spider,Abarth 1.4 Scorpione 170,36.511€,2 puertas,,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,,Gasolina,148 gr/km,1.368 cc,Tracción trasera,45 lts.,Manual,"703,1 km",6,Tracción trasera,232 km/h,"6,80 s.",170 cv,"6,4 l/100km",5000 rpm,250Nm,0,Sí,No,NaN,NaN,,No,No,No,No,NaN,No,,NaN,No/No/No,No,Roadster,4054/1740/1233,2,1135 kg,NaN
3,Abarth,124 Spider,Abarth 1.4 Aut. 170,37.961€,2 puertas,,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,,Gasolina,153 gr/km,1.368 cc,Tracción trasera,45 lts.,Automática secuencial,"681,8 km",6,Tracción trasera,229 km/h,"6,90 s.",170 cv,"6,6 l/100km",5000 rpm,250Nm,0,Sí,No,NaN,NaN,,No,No,No,No,NaN,No,,NaN,No/No/No,No,Roadster,4054/1740/1233,2,1155 kg,NaN
4,Abarth,124 Spider,Abarth 1.4 Scorpione Aut. 170,38.511€,2 puertas,,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,,Gasolina,153 gr/km,1.368 cc,Tracción trasera,45 lts.,Automática secuencial,"681,8 km",6,Tracción trasera,229 km/h,"6,90 s.",170 cv,"6,6 l/100km",5000 rpm,250Nm,0,Sí,No,NaN,NaN,,No,No,No,No,NaN,No,,NaN,No/No/No,No,Roadster,4054/1740/1233,2,1155 kg,NaN
5,Abarth,595,Abarth 1.4T,Jet 145,15.670€,3 puertas,,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,0 meses,,Gasolina,139 gr/km,1.368 cc,Tracción delantera,35 lts.,Manual,"583,3 km",5,Tracción delantera,205 km/h,"7,90 s.",145 cv,"6,0 l/100km",5500 rpm,206Nm,0,Sí,No,NaN,NaN,,No,No,No,No,NaN,No,,NaN,No/No/No,No,Berlina,3660/1627/1480,3,1110 kg


In [79]:
colnames = [
    'marca',
    'modelo',
    'version',
    'precio',
    'puertas',
    'desc',
    'combustible',
    'traccion',
    'cv',
    'cambio',
    'n_marchas',
    'co2',
    'autonomia',
    'consumo',
    'airbags',
    'garantia',
    'desc4',
    'comb',
    'co2',
    'cilindrada',
    'traccion',
    'capacidad deposito',
    'cambio',
    'autonomia',
    'marchas',
    'traccion',
    'max_vel',
    '0_a_100',
    'cv',
    'consumo',
    'rpm',
    'par',
    'airbags',
    'control traccion',
    'control estabilidad',
    'estrellas NCAP',
    'desc9',
    'inmovilizador',
    'alarma',
    'cierre centralizado',
    'cierre automatico',
    'tipo llave',
    'start/stop',
    'desc16',
    'n_altavoces',
    'desc18',
    'CD/mp3/dvd',
    'cargador',
    'carroceria',
    'medidas largo/ancho/alto',
    'puertas',
    'peso',
    'desc24'
]

In [80]:
pd.set_option('display.max_columns', None)

In [81]:
colnames_to_delete = [
    'desc',
    'desc4',
    'comb',
    'traccion.1',
    'cambio.1',
    'autonomia.1',
    'cv.1',
    'co2.1',
    'consumo.1',
    'traccion.2',
    'inmovilizador',
    'airbags.1',
    'desc9',
    'desc16',
    'n_altavoces',
    'desc18',
    'cargador',
    'medidas largo/ancho/alto',
    'puertas.1',
    'desc24'
]

In [82]:
df_1 = df_1.drop(labels=colnames_to_delete,axis=1)

In [83]:
df.shape

(5927, 1)

In [84]:
df_1.head()

,marca,modelo,version,precio,puertas,combustible,traccion,cv,cambio,n_marchas,co2,autonomia,consumo,airbags,garantia,cilindrada,capacidad deposito,marchas,max_vel,0_a_100,rpm,par,control traccion,control estabilidad,estrellas NCAP,alarma,cierre centralizado,cierre automatico,tipo llave,start/stop,CD/mp3/dvd,carroceria,peso
1,Abarth,124 Spider,Abarth 1.4 170,35.093€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg
2,Abarth,124 Spider,Abarth 1.4 Scorpione 170,36.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Manual,6 marchas,148 gr/km,"703,1 km","6,4 l/100km",0,24 meses,1.368 cc,45 lts.,6,232 km/h,"6,80 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1135 kg
3,Abarth,124 Spider,Abarth 1.4 Aut. 170,37.961€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg
4,Abarth,124 Spider,Abarth 1.4 Scorpione Aut. 170,38.511€,2 puertas,Gasolina,Tracción trasera,170 cv,Automática secuencial,6 marchas,153 gr/km,"681,8 km","6,6 l/100km",0,24 meses,1.368 cc,45 lts.,6,229 km/h,"6,90 s.",5000 rpm,250Nm,Sí,No,NaN,No,No,No,No,NaN,No/No/No,Roadster,1155 kg
5,Abarth,595,Abarth 1.4T,Jet 145,15.670€,,Gasolina,Tracción delantera,145 cv,Manual,5 marchas,139 gr/km,"583,3 km","6,0 l/100km",0,139 gr/km,Tracción delantera,"583,3 km",Tracción delantera,205 km/h,"6,0 l/100km",5500 rpm,0,Sí,No,,No,No,No,No,NaN,No,3


In [85]:
df_1.to_csv('Dataframe_limpio.csv', encoding='utf-8')

## Scrapear los mejores precios - oferta de concesionarios    

In [4]:
import requests as rq
from bs4 import BeautifulSoup

main_page = rq.get("http://www.autodescuento.com")
main_soup = BeautifulSoup(main_page.content, 'html.parser')
path_marcas = "http://www.autodescuento.com/coches-nuevos-"

lista_marcas = []

for marca in main_soup.find_all('h2', class_='h2'):
    
    lista_marcas.append(marca.get_text().strip().replace(' ','-'))

lista_marcas_limpia = lista_marcas[1:-2]


precios_modelos = []

for marca in lista_marcas_limpia:
    
    second_page = rq.get(path_marcas+marca+".htm", headers={'User-Agent': 'Safari'})
    second_soup = BeautifulSoup(second_page.content, 'html.parser')
    
    for modelo in second_soup.find('div', class_='izquierda').find_all('a', class_="coche redondas"):

        model = modelo.find('p',class_='mm').getText().encode('utf-8') + "-"
        third_page = rq.get(modelo['href'], headers={'User-Agent': 'Safari'})
        third_soup = BeautifulSoup(third_page.content, 'html.parser')
        
        if(third_soup.find('span',id='tdversiones') is not None):

            for ver in third_soup.find('span',id='tdversiones').find_all('a', href=True):
                
                precios_modelos.append(ver.find('div', class_='precio').getText())
                #print(ver.find('div', class_='precio').getText())
                
                



35.093€
36.511€
37.961€
38.511€
15.670€
17.316€
20.497€
21.086€
21.861€
22.208€
22.936€
23.272€
23.711€
23.786€
24.983€
25.086€
25.636€
25.961€
26.936€
27.491€
27.811€
27.885€
28.086€
29.416€
29.491€
29.736€
29.936€
31.186€
31.415€
33.111€
33.186€
35.111€
14.426€
15.296€
15.471€
15.565€
15.613€
15.761€
15.784€
15.794€
15.836€
15.893€
16.032€
16.078€
16.091€
16.193€
16.302€
16.455€
16.497€
16.507€
16.509€
16.545€
16.572€
16.649€
16.767€
16.897€
16.907€
17.000€
17.008€
17.023€
17.068€
17.087€
17.155€
17.184€
17.316€
17.317€
17.363€
17.380€
17.431€
17.444€
17.456€
17.465€
20.395€
20.661€
21.017€
21.186€
21.186€
21.403€
21.403€
22.041€
22.198€
22.259€
22.259€
22.400€
22.480€
22.480€
22.510€
22.528€
22.565€
22.565€
22.590€
22.760€
22.822€
22.946€
22.976€
23.121€
23.121€
23.204€
23.204€
23.421€
23.421€
23.596€
23.631€
23.813€
23.847€
23.905€
23.932€
23.932€
23.971€
24.063€
24.121€
24.139€
26.589€
26.720€
26.883€
26.950€
27.107€
27.464€
27.885€
28.041€
28.063€
28.093€
28.502€
28.527€
28.581€


In [6]:
import pandas as pd

df_precios = pd.DataFrame({'precio':precios_modelos})

In [8]:
df_precios.shape

(5918, 1)

In [16]:
df_precios.to_csv('precios_versiones',encoding='utf-8')